In [1]:
import pymonntorch as pmt

import torch
import matplotlib.pyplot as plt

In [2]:
from timeresolution import Timeresolution
from current import ConstanceCurrent, UniformCurrent, UniformCurrentInOneLine , StepCurrent
from lif import LIF
from inputforward import (
    FullConnectivityFirstOption,
    FullConnectivitySecondOption,
    Scaling,
    FixedAAndFinite,
)
from dandrite import Dandrite
from populationactivity import PopulationActivity
from plot import print_plot

In [3]:
def BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=16, 
                                                        R=1.17, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                   model_excitatory2=LIF(tau=16, 
                                                        R=1.17, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                    model_inhibitory=LIF(tau=0.5,
                                                        R=30,
                                                        u_rest=-67,
                                                        u_reset=-75,
                                                        threshold=-55,
                                                        ratio=1,),
                                    
                                    current_excitatory1=ConstanceCurrent(current=20),
                                    
                                    current_excitatory2=ConstanceCurrent(current=20),
                                    
                                    dt=1,
                                    iteration_num=100,
                                    PLT=True,
                                    size_ecitatory1=80,
                                    size_ecitatory2=80,
                                    size_inhibitory=20,
                                    
                                    Excitatory_To_Inhibitory=FullConnectivitySecondOption(J0=100, standardـdeviation=10),
                                    Excitatory_To_Excitatory=FullConnectivitySecondOption(J0=5, standardـdeviation=10),
                                    Inhibitory_To_Excitatory=FullConnectivitySecondOption(J0=-100, standardـdeviation=10)):
    

    net2 = pmt.Network(behavior={1: Timeresolution(dt=dt)})

    excitatory1ـneuron_group = pmt.NeuronGroup(size_ecitatory1,
                                                net=net2,
                                                tag="excitatoryـneuron_group1",
                                                behavior={
                                                    3: current_excitatory1,
                                                    5: Dandrite(),
                                                    6: model_excitatory1,
                                                    7: PopulationActivity(),
                                                    13: pmt.Recorder(variables=["u", "I", "w", "T"], tag="ng1_rec"),
                                                    14: pmt.EventRecorder(variables=["spike"], tag="ng1_even")})
    
    
    excitatory2ـneuron_group = pmt.NeuronGroup(size_ecitatory2,
                                                net=net2,
                                                tag="excitatoryـneuron_group2",
                                                behavior={
                                                    3: current_excitatory2,
                                                    5: Dandrite(),
                                                    6: model_excitatory2,
                                                    7: PopulationActivity(),
                                                    13: pmt.Recorder(variables=["u", "I", "w", "T"], tag="ng1_rec"),
                                                    14: pmt.EventRecorder(variables=["spike"], tag="ng1_even") })

    inhibitory_neouron_group = pmt.NeuronGroup(size_inhibitory,
                                                tag="inhibitory_neuron_group1",
                                                net=net2,
                                                behavior={
                                                    3: ConstanceCurrent(current=0),
                                                    5: Dandrite(),
                                                    6: model_inhibitory,
                                                    7: PopulationActivity(),
                                                    13: pmt.Recorder(variables=["u", "I", "w", "T"], tag="ng2_rec"),
                                                    14: pmt.EventRecorder(variables=["spike"], tag="ng2_even") })

    syn_from_excitatory1_to_inhibitory = pmt.SynapseGroup(tag="from_excitatory1_to_inhibitory",
                                                        net=net2,
                                                        src=excitatory1ـneuron_group,
                                                        dst=inhibitory_neouron_group,
                                                        behavior={4: Excitatory_To_Inhibitory})
    
    syn_from_excitatory2_to_inhibitory = pmt.SynapseGroup(tag="from_excitatory2_to_inhibitory",
                                                        net=net2,
                                                        src=excitatory2ـneuron_group,
                                                        dst=inhibitory_neouron_group,
                                                        behavior={4: Excitatory_To_Inhibitory})

    syn_from_inhibitory_to_excitatory1 = pmt.SynapseGroup(tag="from_inhibitory_to_excitatory1",
                                                        net=net2,
                                                        src=inhibitory_neouron_group,
                                                        dst=excitatory1ـneuron_group,
                                                        behavior={4: Inhibitory_To_Excitatory})
    
    syn_from_inhibitory_to_excitatory2 = pmt.SynapseGroup(tag="from_inhibitory_to_excitatory1",
                                                        net=net2,
                                                        src=inhibitory_neouron_group,
                                                        dst=excitatory2ـneuron_group,
                                                        behavior={4: Inhibitory_To_Excitatory})
    
    syn_inside_excitatory1 = pmt.SynapseGroup(tag="inside_excitatory1",
                                            net=net2,
                                            src=excitatory1ـneuron_group,
                                            dst=excitatory1ـneuron_group,
                                            behavior={4: Excitatory_To_Excitatory})

    syn_inside_excitatory2 = pmt.SynapseGroup(tag="inside_excitatory2",
                                            net=net2,
                                            src=excitatory2ـneuron_group,
                                            dst=excitatory2ـneuron_group,
                                            behavior={4: Excitatory_To_Excitatory})

    net2.initialize()
    net2.simulate_iterations(iteration_num)

    if PLT:
        print_plot(
            net2,
            ngs=[excitatory1ـneuron_group, excitatory2ـneuron_group, inhibitory_neouron_group],
            print_sum_activities=False,
            scaling_factor=1.2
        )

    return net2

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=20, 
                                                        R=2.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                   model_excitatory2=LIF(tau=20, 
                                                        R=2.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                    model_inhibitory=LIF(tau=10,
                                                        R=5,
                                                        u_rest=-67,
                                                        u_reset=-75,
                                                        threshold=-55,
                                                        ratio=1,),
                                    
                                    current_excitatory1=StepCurrent(t0=2, current0=9, noise_range=1.5),
                                    
                                    current_excitatory2= StepCurrent(t0=2, t1=30, current0=9, current1=15, noise_range=1.5),
                                    
                                    dt=1,
                                    iteration_num=100,
                                    PLT=True,
                                    size_ecitatory1=400,
                                    size_ecitatory2=400,
                                    size_inhibitory=200,
                                    
                                    Excitatory_To_Inhibitory=FixedAAndFinite(C=20, J0=50,standard_deviation=0),
                                    Excitatory_To_Excitatory=FixedAAndFinite(J0=2, C=10),
                                    Inhibitory_To_Excitatory=FixedAAndFinite(C=20, J0=-40))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=20, 
                                                        R=2.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                   model_excitatory2=LIF(tau=20, 
                                                        R=2.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                    model_inhibitory=LIF(tau=10,
                                                        R=5,
                                                        u_rest=-67,
                                                        u_reset=-75,
                                                        threshold=-55,
                                                        ratio=1,),
                                    
                                    current_excitatory1=StepCurrent(t0=2, current0=9, noise_range=1.5),
                                    
                                    current_excitatory2= StepCurrent(t0=2, t1=30, current0=9, current1=15, noise_range=1.5),
                                    
                                    dt=1,
                                    iteration_num=100,
                                    PLT=True,
                                    size_ecitatory1=400,
                                    size_ecitatory2=400,
                                    size_inhibitory=200,
                                    
                                    Excitatory_To_Inhibitory=FixedAAndFinite(C=20, J0=50,standard_deviation=0),
                                    Excitatory_To_Excitatory=FixedAAndFinite(J0=100, C=10),
                                    Inhibitory_To_Excitatory=FixedAAndFinite(C=20, J0=-5))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=20, 
                                                        R=2.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                   model_excitatory2=LIF(tau=20, 
                                                        R=2.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                    model_inhibitory=LIF(tau=10,
                                                        R=5,
                                                        u_rest=-67,
                                                        u_reset=-75,
                                                        threshold=-55,
                                                        ratio=1,),
                                    
                                    current_excitatory1=StepCurrent(t0=2, current0=9, noise_range=1.5),
                                    
                                    current_excitatory2= StepCurrent(t0=2, t1=30, current0=9, current1=15, noise_range=1.5),
                                    
                                    dt=1,
                                    iteration_num=100,
                                    PLT=True,
                                    size_ecitatory1=400,
                                    size_ecitatory2=400,
                                    size_inhibitory=200,
                                    
                                    Excitatory_To_Inhibitory=FixedAAndFinite(C=20, J0=50,standard_deviation=0),
                                    Excitatory_To_Excitatory=FixedAAndFinite(J0=2, C=10),
                                    Inhibitory_To_Excitatory=FixedAAndFinite(C=20, J0=-5))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=20, 
                                                        R=2.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                   model_excitatory2=LIF(tau=20, 
                                                        R=2.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                    model_inhibitory=LIF(tau=3,
                                                        R=70,
                                                        u_rest=-67,
                                                        u_reset=-75,
                                                        threshold=-55,
                                                        ratio=1,),
                                    
                                    current_excitatory1=StepCurrent(t0=2, current0=9, noise_range=1.5),
                                    
                                    current_excitatory2= StepCurrent(t0=2, t1=30, current0=9, current1=15, noise_range=1.5),
                                    
                                    dt=1,
                                    iteration_num=100,
                                    PLT=True,
                                    size_ecitatory1=400,
                                    size_ecitatory2=400,
                                    size_inhibitory=200,
                                    
                                    Excitatory_To_Inhibitory=FixedAAndFinite(C=20, J0=10,standard_deviation=0),
                                    Excitatory_To_Excitatory=FixedAAndFinite(J0=2, C=10),
                                    Inhibitory_To_Excitatory=FixedAAndFinite(C=20, J0=-5))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=20, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_excitatory2=LIF(tau=20, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_inhibitory=LIF(tau=3,
                                                    R=70,
                                                    u_rest=-67,
                                                    u_reset=-75,
                                                    threshold=-55,
                                                    ratio=1,),
                                
                                current_excitatory1=StepCurrent(t0=5, current=11.5, current0=11.5, noise_range=0.5),
                                
                                current_excitatory2= StepCurrent(t0=5, t1=30, current=11.5, current0=11.5,  current1=15, noise_range=0.5),
                                
                                dt=1,
                                iteration_num=100,
                                PLT=True,
                                size_ecitatory1=400,
                                size_ecitatory2=400,
                                size_inhibitory=80,
                                
                                Excitatory_To_Inhibitory=FixedAAndFinite(C=5, J0=15,standard_deviation=20),
                                Excitatory_To_Excitatory=FixedAAndFinite(J0=10, C=10, standard_deviation=20),
                                Inhibitory_To_Excitatory=FixedAAndFinite(C=30, J0=-12, standard_deviation=20))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_excitatory2=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_inhibitory=LIF(tau=3,
                                                    R=70,
                                                    u_rest=-67,
                                                    u_reset=-75,
                                                    threshold=-55,
                                                    ratio=1,),

                                current_excitatory1=StepCurrent(t0=10,current=10.5, current0=11.5, current1=11.5, noise_range=0.5),
                                
                                current_excitatory2=StepCurrent(t0=10, t1=30, current=10.5, current0=11.5,  current1=15, noise_range=0.5),
                                
                                dt=1,
                                iteration_num=100,
                                PLT=True,
                                size_ecitatory1=800,
                                size_ecitatory2=800,
                                size_inhibitory=200,
                                

                                Excitatory_To_Inhibitory=FixedAAndFinite(C=10, J0=50,standard_deviation=20),
                                Excitatory_To_Excitatory=FixedAAndFinite(J0=20, C=10, standard_deviation=20),
                                Inhibitory_To_Excitatory=FixedAAndFinite(C=30, J0=-9, standard_deviation=20))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_excitatory2=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_inhibitory=LIF(tau=3,
                                                    R=70,
                                                    u_rest=-67,
                                                    u_reset=-75,
                                                    threshold=-55,
                                                    ratio=1,),
                                
                                current_excitatory1=StepCurrent(t0=10,current=10.5, current0=11.5, current1=11.5, noise_range=0.5),
                                
                                current_excitatory2= StepCurrent(t0=10, t1=30, current=10.5, current0=11.5,  current1=15, noise_range=0.5),
                                
                                dt=1,
                                iteration_num=100,
                                PLT=True,
                                size_ecitatory1=800,
                                size_ecitatory2=800,
                                size_inhibitory=200,
                                

                                Excitatory_To_Inhibitory=Scaling(p=0.01, J0=10,standard_deviation=75),
                                Excitatory_To_Excitatory=FixedAAndFinite(J0=10, C=10, standard_deviation=75),
                                Inhibitory_To_Excitatory=Scaling(p=0.55, J0=-12, standard_deviation=75))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_excitatory2=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_inhibitory=LIF(tau=3,
                                                    R=70,
                                                    u_rest=-67,
                                                    u_reset=-75,
                                                    threshold=-55,
                                                    ratio=1,),
                                
                                current_excitatory1=StepCurrent(t0=10,current=10.5, current0=11.5, current1=11.5, noise_range=0.5),
                                
                                current_excitatory2= StepCurrent(t0=10, t1=30, current=10.5, current0=11.5,  current1=15, noise_range=0.5),
                                
                                dt=1,
                                iteration_num=100,
                                PLT=True,
                                size_ecitatory1=800,
                                size_ecitatory2=800,
                                size_inhibitory=200,
                                

                                Excitatory_To_Inhibitory=Scaling(p=0.9, J0=10,standard_deviation=75),
                                Excitatory_To_Excitatory=FixedAAndFinite(J0=10, C=10, standard_deviation=75),
                                Inhibitory_To_Excitatory=Scaling(p=0.55, J0=-12, standard_deviation=75))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_excitatory2=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_inhibitory=LIF(tau=3,
                                                    R=70,
                                                    u_rest=-67,
                                                    u_reset=-75,
                                                    threshold=-55,
                                                    ratio=1,),
                                
                                current_excitatory1=StepCurrent(t0=10,current=10.5, current0=11.5, current1=11.5, noise_range=0.5),
                                
                                current_excitatory2= StepCurrent(t0=10, t1=30, current=10.5, current0=11.5,  current1=15, noise_range=0.5),
                                
                                dt=1,
                                iteration_num=100,
                                PLT=True,
                                size_ecitatory1=800,
                                size_ecitatory2=800,
                                size_inhibitory=200,
                                

                                Excitatory_To_Inhibitory=FixedAAndFinite(C=10, J0=50,standard_deviation=20),
                                Excitatory_To_Excitatory=FullConnectivityFirstOption(J0=20),
                                Inhibitory_To_Excitatory=Scaling(p=0.01, J0=-12, standard_deviation=75))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_excitatory2=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_inhibitory=LIF(tau=3,
                                                    R=70,
                                                    u_rest=-67,
                                                    u_reset=-75,
                                                    threshold=-55,
                                                    ratio=1,),
                                
                                current_excitatory1=StepCurrent(t0=10,current=10.5, current0=11.5, current1=11.5, noise_range=0.5),
                                
                                current_excitatory2= StepCurrent(t0=10, t1=30, current=10.5, current0=11.5,  current1=15, noise_range=0.5),
                                
                                dt=1,
                                iteration_num=100,
                                PLT=True,
                                size_ecitatory1=800,
                                size_ecitatory2=800,
                                size_inhibitory=200,
                                

                                Excitatory_To_Inhibitory=FixedAAndFinite(C=10, J0=50,standard_deviation=20),
                                Excitatory_To_Excitatory=FullConnectivityFirstOption(J0=20),
                                Inhibitory_To_Excitatory=Scaling(p=0.9, J0=-12, standard_deviation=75))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_excitatory2=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_inhibitory=LIF(tau=3,
                                                    R=70,
                                                    u_rest=-67,
                                                    u_reset=-75,
                                                    threshold=-55,
                                                    ratio=1,),
                                
                                current_excitatory1=StepCurrent(t0=10,current=10.5, current0=11.5, current1=11.5, noise_range=0.5),
                                
                                current_excitatory2= StepCurrent(t0=10, t1=30, current=10.5, current0=11.5,  current1=15, noise_range=0.5),
                                
                                dt=1,
                                iteration_num=100,
                                PLT=True,
                                size_ecitatory1=800,
                                size_ecitatory2=800,
                                size_inhibitory=200,
                                

                                Excitatory_To_Inhibitory=FullConnectivityFirstOption(J0=50),
                                Excitatory_To_Excitatory=FullConnectivityFirstOption(J0=20),
                                Inhibitory_To_Excitatory=FullConnectivitySecondOption(J0=-12, standard_deviation=75))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_excitatory2=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_inhibitory=LIF(tau=3,
                                                    R=70,
                                                    u_rest=-67,
                                                    u_reset=-75,
                                                    threshold=-55,
                                                    ratio=1,),
                                
                                current_excitatory1=StepCurrent(t0=10,current=10.5, current0=11.5, current1=11.5, noise_range=0.5),
                                
                                current_excitatory2= StepCurrent(t0=10, t1=30, current=10.5, current0=11.5,  current1=15, noise_range=0.5),
                                
                                dt=1,
                                iteration_num=100,
                                PLT=True,
                                size_ecitatory1=100,
                                size_ecitatory2=100,
                                size_inhibitory=20,
                                

                                Excitatory_To_Inhibitory=FixedAAndFinite(C=10, J0=50,standard_deviation=20),
                                Excitatory_To_Excitatory=FullConnectivityFirstOption(J0=20),
                                Inhibitory_To_Excitatory=Scaling(p=0.01, J0=-12, standard_deviation=75))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_excitatory2=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_inhibitory=LIF(tau=3,
                                                    R=70,
                                                    u_rest=-67,
                                                    u_reset=-75,
                                                    threshold=-55,
                                                    ratio=1,),
                                
                                current_excitatory1=StepCurrent(t0=10,current=10.5, current0=11.5, current1=11.5, noise_range=0.5),
                                
                                current_excitatory2= StepCurrent(t0=10, t1=30, current=10.5, current0=11.5,  current1=15, noise_range=0.5),
                                
                                dt=1,
                                iteration_num=100,
                                PLT=True,
                                size_ecitatory1=100,
                                size_ecitatory2=100,
                                size_inhibitory=20,
                                

                                Excitatory_To_Inhibitory=FixedAAndFinite(C=10, J0=50,standard_deviation=20),
                                Excitatory_To_Excitatory=FullConnectivityFirstOption(J0=20),
                                Inhibitory_To_Excitatory=Scaling(p=0.9, J0=-12, standard_deviation=75))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_excitatory2=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_inhibitory=LIF(tau=3,
                                                    R=70,
                                                    u_rest=-67,
                                                    u_reset=-75,
                                                    threshold=-55,
                                                    ratio=1,),
                                
                                current_excitatory1=StepCurrent(t0=10,current=10.5, current0=11.5, current1=11.5, noise_range=0.5),
                                
                                current_excitatory2= StepCurrent(t0=10, t1=30, current=10.5, current0=11.5,  current1=15, noise_range=0.5),
                                
                                dt=1,
                                iteration_num=100,
                                PLT=True,
                                size_ecitatory1=100,
                                size_ecitatory2=100,
                                size_inhibitory=20,
                                

                                Excitatory_To_Inhibitory=FullConnectivityFirstOption(J0=50),
                                Excitatory_To_Excitatory=FullConnectivityFirstOption(J0=20),
                                Inhibitory_To_Excitatory=FullConnectivitySecondOption(J0=-12, standard_deviation=75))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_excitatory2=LIF(tau=25, 
                                                    R=1.2, 
                                                    u_rest=-67, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_inhibitory=LIF(tau=3,
                                                    R=70,
                                                    u_rest=-67,
                                                    u_reset=-75,
                                                    threshold=-55,
                                                    ratio=1,),
                                
                                current_excitatory1=StepCurrent(t0=10,current=10.5, current0=11.5, current1=11.5, noise_range=0.5),
                                
                                current_excitatory2= StepCurrent(t0=10, t1=30, current=10.5, current0=11.5,  current1=15, noise_range=0.5),
                                
                                dt=1,
                                iteration_num=100,
                                PLT=True,
                                size_ecitatory1=100,
                                size_ecitatory2=100,
                                size_inhibitory=20,
                                

                                Excitatory_To_Inhibitory=Scaling(p=0.9, J0=10,standard_deviation=75),
                                Excitatory_To_Excitatory=FixedAAndFinite(J0=10, C=10, standard_deviation=75),
                                Inhibitory_To_Excitatory=Scaling(p=0.55, J0=-12, standard_deviation=75))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=20, 
                                                    R=1.7, 
                                                    u_rest=-65, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_excitatory2=LIF(tau=20, 
                                                    R=1.7, 
                                                    u_rest=-65, 
                                                    u_reset=-75, 
                                                    threshold=-55, 
                                                    ratio=1),
                                
                                model_inhibitory=LIF(tau=3,
                                                    R=70,
                                                    u_rest=-65,
                                                    u_reset=-75,
                                                    threshold=-55,
                                                    ratio=1,),
                                
                                current_excitatory1=StepCurrent(t0=10,current=10.5, current0=11.5, current1=11.5, noise_range=0.5),
                                
                                current_excitatory2= StepCurrent(t0=10, t1=30, current=6.4, current0=6.7,  current1=9, noise_range=0.5),
                                
                                dt=1,
                                iteration_num=100,
                                PLT=True,
                                size_ecitatory1=800,
                                size_ecitatory2=800,
                                size_inhibitory=200,
                                
                                Excitatory_To_Inhibitory=FixedAAndFinite(C=8, J0=20, standard_deviation=20),
                                Excitatory_To_Excitatory=FixedAAndFinite(J0=25, C=40, standard_deviation=20),
                                Inhibitory_To_Excitatory=FixedAAndFinite(C=500, J0=-7, standard_deviation=20))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=20, 
                                                        R=2.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                   model_excitatory2=LIF(tau=20, 
                                                        R=2.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                    model_inhibitory=LIF(tau=10,
                                                        R=5,
                                                        u_rest=-67,
                                                        u_reset=-75,
                                                        threshold=-55,
                                                        ratio=1,),
                                    
                                    current_excitatory1=StepCurrent(t0=2, current0=9, noise_range=1.5),
                                    
                                    current_excitatory2= StepCurrent(t0=2, current0=9, noise_range=1.5),
                                    
                                    dt=1,
                                    iteration_num=100,
                                    PLT=True,
                                    size_ecitatory1=400,
                                    size_ecitatory2=400,
                                    size_inhibitory=200,
                                    
                                    Excitatory_To_Inhibitory=FixedAAndFinite(C=20, J0=50,standard_deviation=0),
                                    Excitatory_To_Excitatory=FixedAAndFinite(J0=100, C=10),
                                    Inhibitory_To_Excitatory=FixedAAndFinite(C=20, J0=-40))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=20, 
                                                        R=2.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                   model_excitatory2=LIF(tau=20, 
                                                        R=2.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                    model_inhibitory=LIF(tau=6,
                                                        R=20,
                                                        u_rest=-67,
                                                        u_reset=-75,
                                                        threshold=-55,
                                                        ratio=1,),
                                    
                                    current_excitatory1=StepCurrent(t0=2, current0=9, noise_range=1.5),
                                    
                                    current_excitatory2= StepCurrent(t0=2, t1=30, current0=9, current1=11, noise_range=1.5),
                                    
                                    dt=1,
                                    iteration_num=100,
                                    PLT=True,
                                    size_ecitatory1=400,
                                    size_ecitatory2=400,
                                    size_inhibitory=200,
                                    
                                    Excitatory_To_Inhibitory=FixedAAndFinite(C=8, J0=30,standard_deviation=0),
                                    Excitatory_To_Excitatory=FixedAAndFinite(J0=25, C=10),
                                    Inhibitory_To_Excitatory=FixedAAndFinite(C=10, J0=-12))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=15, 
                                                        R=2.5, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                   model_excitatory2=LIF(tau=15, 
                                                        R=2.5, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                    model_inhibitory=LIF(tau=15,
                                                        R=15,
                                                        u_rest=-67,
                                                        u_reset=-75,
                                                        threshold=-55,
                                                        ratio=1,),
                                    
                                    current_excitatory1=UniformCurrentInOneLine(current=11 , initial_current = 120,step = 5),
                                    
                                    current_excitatory2=UniformCurrentInOneLine(current=15 , initial_current = 120,step = 5),
                                    
                                    dt=1,
                                    iteration_num=100,
                                    PLT=True,
                                    size_ecitatory1=40,
                                    size_ecitatory2=40,
                                    size_inhibitory=20,
                                    
                                    Excitatory_To_Inhibitory=FixedAAndFinite(C=30, J0=25),
                                    Excitatory_To_Excitatory=FixedAAndFinite(C=20, J0=10),
                                    Inhibitory_To_Excitatory=FixedAAndFinite(C=40, J0=-50))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=20, 
                                                        R=1.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                   model_excitatory2=LIF(tau=20, 
                                                        R=1.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                    model_inhibitory=LIF(tau=5,
                                                        R=3,
                                                        u_rest=-67,
                                                        u_reset=-75,
                                                        threshold=-55,
                                                        ratio=1,),
                                    
                                    current_excitatory1=UniformCurrent(current=8.75 , noise_range=1),
                                    
                                    current_excitatory2=UniformCurrent(current=9 , noise_range=1),
                                    
                                    dt=1,
                                    iteration_num=100,
                                    PLT=True,
                                    size_ecitatory1=400,
                                    size_ecitatory2=400,
                                    size_inhibitory=200,
                                    
                                    Excitatory_To_Inhibitory=FixedAAndFinite(C=5, J0=100),
                                    Excitatory_To_Excitatory=FixedAAndFinite(C=5, J0=100),
                                    Inhibitory_To_Excitatory=Scaling(p=1, J0=-40))

In [ ]:
BuildNeuronNetwork_NeuronGroup( model_excitatory1=LIF(tau=25, 
                                                        R=1.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                   model_excitatory2=LIF(tau=25, 
                                                        R=1.2, 
                                                        u_rest=-67, 
                                                        u_reset=-75, 
                                                        threshold=-55, 
                                                        ratio=1),
                                   
                                    model_inhibitory=LIF(tau=5,
                                                        R=3,
                                                        u_rest=-67,
                                                        u_reset=-75,
                                                        threshold=-55,
                                                        ratio=1,),
                                    
                                    current_excitatory1=UniformCurrent(current=8.80 , noise_range=1),
                                    
                                    current_excitatory2= UniformCurrent(current=9 , noise_range=1),
                                    
                                    dt=1,
                                    iteration_num=100,
                                    PLT=True,
                                    size_ecitatory1=400,
                                    size_ecitatory2=400,
                                    size_inhibitory=200,
                                    
                                    Excitatory_To_Inhibitory=FixedAAndFinite(C=10, J0=90,standard_deviation=0),
                                    Excitatory_To_Excitatory=FullConnectivityFirstOption(J0=1000),
                                    Inhibitory_To_Excitatory=FixedAAndFinite(C=10, J0=-90))